In [2]:
import imageio
import itk
import matplotlib.pyplot as plt
import numpy as np
import skimage

In [3]:
def reg2d(fixed_image, moving_image):
    fixed_image = itk.GetImageFromArray(fixed_image.astype(np.float32))
    moving_image = itk.GetImageFromArray(moving_image.astype(np.float32))
    
    dimension = fixed_image.GetImageDimension()
    FixedImageType = type(fixed_image)
    MovingImageType = type(moving_image)

    transform = itk.Rigid2DTransform[itk.D].New()

    TransformInitializerType = itk.CenteredTransformInitializer[
        itk.MatrixOffsetTransformBase[itk.D, dimension, dimension],
        FixedImageType,
        MovingImageType,
    ]
    initializer = TransformInitializerType.New(
        Transform=transform,
        FixedImage=fixed_image,
        MovingImage=moving_image,
    )
    initializer.InitializeTransform()

    metric = itk.MeanSquaresImageToImageMetricv4[FixedImageType, MovingImageType].New()

    optimizer = itk.RegularStepGradientDescentOptimizerv4.New(
        LearningRate=4,
        MinimumStepLength=0.001,
        RelaxationFactor=0.5,
        NumberOfIterations=1000,
    )
    
    scales = optimizer.GetScales()
    scales.SetSize(3)
    scales.SetElement(0, 200.0) # 1rad rotation has the same weight as 200mm translation
    scales.SetElement(1, 1.0)
    scales.SetElement(2, 1.0)
    optimizer.SetScales(scales)

    RegistrationType = itk.ImageRegistrationMethodv4[FixedImageType, MovingImageType]
    registration = RegistrationType.New(
        Metric=metric,
        Optimizer=optimizer,
        FixedImage=fixed_image,
        MovingImage=moving_image,
        InitialTransform=transform,
        NumberOfLevels=1,
    )

    registration.Update()

    finalParameters = transform.GetParameters()
    finalParameters = itk.array_from_vnl_vector(transform.GetParameters())

    numberOfIterations = optimizer.GetCurrentIteration()
    bestValue = optimizer.GetValue()

    return finalParameters, numberOfIterations, bestValue

In [4]:
fixed_image = imageio.imread('BrainProtonDensitySliceBorder20.png')
moving_image = imageio.imread('BrainProtonDensitySliceShifted13x17y.png')

reg2d(fixed_image, moving_image)

(array([5.08737259e-04, 1.29995933e+01, 1.69999615e+01]),
 40,
 0.03739409090253444)

In [5]:
def add_sphere(location, radius, img):
    rr, cc = skimage.draw.ellipse(location[0], location[1],
                                  radius, radius,
                                  img.shape)
    img[rr, cc] = True

def spheres(info, shape):
    img = np.zeros(shape, np.bool_)
    for location, radius in info:
        add_sphere(location, radius, img)
    return img
        
def reg(cavity_info, atom_info, shape):
    cavity = spheres(cavity_info, shape)
    atoms = spheres(atom_info, shape)
    return cavity, atoms, reg2d(cavity, atoms)

In [6]:
c_info = [((15,20), 6), ((20, 20), 6)]
a_info = [((15,22), 6), ((20, 22), 6)]

cavity, atoms, results = reg(c_info, a_info, (50,50))

In [7]:
results

(array([0.03305866, 1.75683498, 0.15206176]), 11, 3.126112133537907e-05)